In [1]:
import torch
from models.models import *
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

/home/server-ailab-12gb/miniconda3/envs/Khoa_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'cuda'

In [2]:
# architecture = "TSF"
# architecture = "TSF_hard_voting"
# architecture = "TSF_Resnet_Swin"
# architecture = "TSF_ViT_Resnet"
architecture = "TSF_ViT_Swin"
# architecture = "Baseline"
# architecture = "Baseline_hard_voting"
# architecture = "Baseline_Resnet_Swin"
# architecture = "Baseline_ViT_Resnet"
# architecture = "Baseline_ViT_Swin"
# architecture = "TSF_Resnet"
# architecture = "TSF_ViT"
# architecture = "TSF_Swin"
train_index_checkpoint = ''
test_index_checkpoint = '12'
gamma = 2
alpha = 0.3
version = architecture + "_" + str(gamma) + "_" + str(alpha)

In [3]:
# FOLDER = "/home/server-ailab-12gb/DUC-MMM"
FOLDER = "/home/server-ailab-12gb/DUC-MMM"
data_dir = '/home/server-ailab-12gb/DUC-MMM/data/Musti'
train_dir = os.path.join(data_dir, 'train')
test_dir = os.path.join(data_dir, 'test')
train_img_dir = os.path.join(train_dir, 'img')
train_file_path = os.path.join(train_dir, 'train.json')

test_img_dir = os.path.join(test_dir, 'img')
test_file_path = os.path.join(test_dir, 'test.json')
history_save_folder = os.path.join(FOLDER, 'history_save')
checkpoint_dir = os.path.join(FOLDER, 'checkpoint')
checkpoint_directory = os.path.join(checkpoint_dir, version)


predict_dir = os.path.join(FOLDER, "predict")
predict_path = os.path.join(predict_dir, version + '.json')
report_dir = os.path.join(FOLDER, "report")
report_path = os.path.join(report_dir, version)

report_dir = os.path.join(FOLDER, "report")
report_path = os.path.join(report_dir, version)

lr = 0.001
epoch = 0
loss = 10000

In [4]:
image_processor = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224-in21k")
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
vit_encoder = ViTModel.from_pretrained("google/vit-base-patch16-224-in21k")
resnet_encoder = ResNetForImageClassification.from_pretrained("microsoft/resnet-50")
resnet_encoder = nn.Sequential(*list(resnet_encoder.children())[:-1])
swintransformer = AutoModelForImageClassification.from_pretrained("microsoft/swin-tiny-patch4-window7-224")
swintransformer = nn.Sequential(*list(swintransformer.children())[:-1])
text_encoder = BertModel.from_pretrained('bert-base-multilingual-cased')
encoder_layer = text_encoder.encoder
text_embedding= text_encoder.embeddings
pooling_layer= text_encoder.pooler

In [5]:
dataset_test = Dataset_MMM(test_file_path, test_img_dir,image_processor,tokenizer)
dataloader_test = DataLoader(dataset_test,batch_size=16,drop_last=False, shuffle=False)

/home/server-ailab-12gb/DUC-MMM/data/Musti/test/img
Len: 814


In [6]:
if len(test_index_checkpoint) > 0:
    img_encoder =[vit_encoder,resnet_encoder,swintransformer]
    model = get_model(architecture, img_encoder,encoder_layer,text_embedding, pooling_layer)
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    PATH = os.path.join(checkpoint_directory, test_index_checkpoint + CHECKPOINT_EXTENSION)
    epoch, model, optimizer, loss = load_checkpoint(PATH, model, optimizer)
    print("Load checkpoint: "+str(PATH))
list = predict(model,device,dataloader_test,image_processor)
json_dict = read_json(test_file_path)
for i in range(len(json_dict['pairs'])):
    if list[i] == 0:
        json_dict['pairs'][i]['subtask1_label'] = 'NO'
    else:
        json_dict['pairs'][i]['subtask1_label'] = 'YES'
with open(predict_path, 'w') as f:
    json.dump(json_dict, f, indent=4)

Load checkpoint: /home/server-ailab-12gb/DUC-MMM/checkpoint/TSF_ViT_Swin_2_0.3/12.pt


Evaluate:   2%|▏         | 1/51 [00:01<00:52,  1.05s/it]

logitstensor([0.3978, 0.3978, 0.3978, 0.3978, 0.3978, 0.3978, 0.3978, 0.3978, 0.3978,
        0.3978, 0.3978, 0.3978, 0.3978, 0.3978, 0.3978, 0.3978],
       device='cuda:0')


Evaluate:   4%|▍         | 2/51 [00:01<00:28,  1.71it/s]

logitstensor([0.3978, 0.3978, 0.3978, 0.3978, 0.3978, 0.3978, 0.3978, 0.3978, 0.3978,
        0.3978, 0.3978, 0.3978, 0.3978, 0.3978, 0.3978, 0.3978],
       device='cuda:0')


...

logitstensor([0.3978, 0.3978, 0.3978, 0.3978, 0.3978, 0.3978, 0.3978, 0.3978, 0.3978,
        0.3978, 0.3978, 0.3978, 0.3978, 0.3978, 0.3978, 0.3978],
       device='cuda:0')


Evaluate:  98%|█████████▊| 50/51 [00:13<00:00,  3.88it/s]

logitstensor([0.3978, 0.3978, 0.3978, 0.3978, 0.3978, 0.3978, 0.3978, 0.3978, 0.3978,
        0.3978, 0.3978, 0.3978, 0.3978, 0.3978, 0.3978, 0.3978],
       device='cuda:0')


Evaluate: 100%|██████████| 51/51 [00:14<00:00,  3.64it/s]

logitstensor([0.3978, 0.3978, 0.3978, 0.3978, 0.3978, 0.3978, 0.3978, 0.3978, 0.3978,
        0.3978, 0.3978, 0.3978, 0.3978, 0.3978], device='cuda:0')
